<a href="https://colab.research.google.com/github/rahulkumar1502/ERA-V2/blob/main/Assignment-2%5CERA_V2_S2_Rahul_Kumar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# torch: Core PyTorch library
import torch
# torch.nn: Building blocks for neural networks
import torch.nn as nn
# torch.nn.functional: Activation functions and utility functions
import torch.nn.functional as F
# torch.optim: Optimization algorithms for training
import torch.optim as optim
# torchvision.datasets: Commonly used datasets
# torchvision.transforms: Image transformations
from torchvision import datasets, transforms
# torchsummary: Visualizing neural network architectures
!pip install torchsummary
from torchsummary import summary

In [2]:
# checks if a CUDA-enabled GPU is available on the system.
# If it is, the code sets the device to use the GPU for computations.
# Otherwise, it sets the device to use the CPU.
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [3]:
# Set batch size
batch_size = 1024

# Training data loader:
train_loader = torch.utils.data.DataLoader(
      # Load MNIST dataset for training
    datasets.MNIST('../data', train=True, download=True,
                   # Apply transformations to each data point
                    transform=transforms.Compose([
                        transforms.ToTensor(), # Convert data to PyTorch tensor
                        transforms.Normalize((0.1307,), (0.3081,)) # Normalize the data
                    ])),
    batch_size=batch_size,  # Set batch size
    shuffle=True) # Shuffle the data for randomness during training

# Testing data loader:
test_loader = torch.utils.data.DataLoader(
     # Load MNIST dataset for testing
    datasets.MNIST('../data', train=False,
                   # Apply the same transformations as the training data loader
                   transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size,  # Set batch size
    shuffle=True)  # Shuffle the data for randomness during testing (usually set to False for testing)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 56939847.14it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1951629.54it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 2738282.59it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2285606.33it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



# Some Notes on our naive model

We are going to write a network based on what we have learnt so far.

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast".

In [4]:
class FirstDNN(nn.Module):
  def __init__(self):
    super(FirstDNN, self).__init__()
    # r_in:1, n_in:28, j_in:1, s:1, r_out:3, n_out:28, j_out:1
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
    # r_in:3 , n_in:32 , j_in: 1, s:1 , r_out: 5, n_out:28 , j_out:1
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    # r_in:5 , n_in:64 , j_in: , s:1 , r_out: , n_out:64 , j_out:
    self.pool1 = nn.MaxPool2d(2, 2)
    # r_in: , n_in:64 , j_in: , s:2 , r_out: , n_out:128 , j_out:
    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    # r_in: , n_in: , j_in: , s:1 , r_out: , n_out: , j_out:
    self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
    # r_in: , n_in: , j_in: , s:1 , r_out: , n_out: , j_out:
    self.pool2 = nn.MaxPool2d(2, 2)
    # r_in: , n_in: , j_in: , s:2 , r_out: , n_out: , j_out:
    self.conv5 = nn.Conv2d(256, 512, 3)
    # r_in: , n_in: , j_in: , s:1 , r_out: , n_out: , j_out:
    self.conv6 = nn.Conv2d(512, 1024, 3)
    # r_in: , n_in: , j_in: , s:1 , r_out: , n_out: , j_out:
    self.conv7 = nn.Conv2d(1024, 10, 3)
    # Added fully connected layer
    self.fc1 = nn.Linear(10, 10)


# Correct values
# https://user-images.githubusercontent.com/498461/238034116-7db4cec0-7738-42df-8b67-afa971428d39.png
  def forward(self, x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    #x = self.conv7(x)
    x = F.relu(self.conv7(x))
    #x = F.relu(x) # this is the last step. Think what ReLU does to our results at this stage!
    # After above step, all negatives are converted to 0 while remaining tensors are unaffected.
    x = x.view(-1, 10) # reshapes the array x to have a single column with 10 rows
    return F.log_softmax(x)
    # The softmax function is a common activation function used in neural networks.
    # It takes a vector of real values as input and normalizes it into a probability distribution consisting of values between 0 and 1 that sum to 1.
    # The log_softmax function calculates the logarithm of the softmax values.
    # This is useful in certain scenarios such as when training neural networks, as it can improve the numerical stability of the model and make it easier to optimize.


In [5]:
model = FirstDNN().to(device)

In [6]:
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

<ipython-input-4-6606815a05df>:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train() # Set the model to training mode
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # Move the input data and target labels to the specified device (GPU in this case)
        optimizer.zero_grad() # Zero out the gradients in the optimizer
        output = model(data) # Forward pass: compute predicted outputs by passing input data through the model
        loss = F.nll_loss(output, target) # Calculate the negative log likelihood (NLL) loss between the predicted output and target labels
        loss.backward()  # Backward pass: compute gradient of the loss with respect to model parameters
        optimizer.step() # Update the weights using the optimizer
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval() # Set the model to evaluation mode
    # Initialize variables to keep track of test loss and correct predictions
    test_loss = 0
    correct = 0
    # Disable gradient computation during testing
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data) # Forward pass: compute predicted outputs by passing input data through the model
            # Calculate the negative log likelihood (NLL) loss between the predicted output and target labels
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # Sum up batch loss (reduction='sum') for later calculation of average loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset) # Calculate average test loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/59 [00:00<?, ?it/s]<ipython-input-4-6606815a05df>:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=1.7841417789459229 batch_id=58: 100%|██████████| 59/59 [31:56<00:00, 32.49s/it]



Test set: Average loss: 1.8140, Accuracy: 2822/10000 (28%)

